In [ ]:
%pip install 

In [1]:
import os
import json
import re
from PIL import Image

# === CONFIG ===
dataset_root = r"D:\School\Semester 8\PENGANTAR DEEP LEARNING\UTS\dataset-F"
output_json_path = os.path.join(dataset_root, "annotations.json")

# === CLASS DEFINITIONS ===
coco_classes = [
    "front_left_door_closed",
    "front_left_door_open",
    "front_right_door_closed",
    "front_right_door_open",
    "hood_closed",
    "hood_open",
    "rear_left_door_closed",
    "rear_left_door_open",
    "rear_right_door_closed",
    "rear_right_door_open"
]

categories = [
    {"id": i + 1, "name": name, "supercategory": "car_part"}
    for i, name in enumerate(coco_classes)
]
category_map = {cat["name"]: cat["id"] for cat in categories}

# === COCO STRUCTURE ===
images = []
annotations = []
image_id = 1
annotation_id = 1

# Regex pattern to match part-state pairs like: front_left_door-opened
part_state_pattern = re.compile(r"(front_left_door|front_right_door|rear_left_door|rear_right_door|hood)-(opened|closed)")

# === PROCESS EACH IMAGE ===
for subfolder in os.listdir(dataset_root):
    folder_path = os.path.join(dataset_root, subfolder)
    if not os.path.isdir(folder_path):
        continue

    for filename in os.listdir(folder_path):
        if not filename.endswith(".png"):
            continue

        full_path = os.path.join(folder_path, filename)

        # Load image size
        with Image.open(full_path) as img:
            width, height = img.size

        # Add image entry
        images.append({
            "id": image_id,
            "file_name": os.path.relpath(full_path, dataset_root).replace("\\", "/"),
            "width": width,
            "height": height
        })

        # Find all part-state pairs in filename
        matches = part_state_pattern.findall(filename)
        if not matches:
            print(f"⚠️ No valid label found in: {filename}")
            continue

        for part, state in matches:
            normalized = f"{part}_{'open' if state == 'opened' else 'closed'}"
            if normalized not in category_map:
                print(f"⚠️ Skipped unknown label: {normalized} in {filename}")
                continue

            annotations.append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_map[normalized],
                "bbox": [0, 0, width, height],  # dummy bbox
                "area": width * height,
                "iscrowd": 0
            })
            annotation_id += 1

        image_id += 1

# === FINAL COCO JSON ===
coco_output = {
    "info": {
        "description": "Vehicle Component State Dataset",
        "version": "1.0",
        "year": 2025
    },
    "licenses": [],
    "images": images,
    "annotations": annotations,
    "categories": categories
}

# === SAVE JSON ===
with open(output_json_path, "w") as f:
    json.dump(coco_output, f, indent=4)

print(f"\n✅ COCO-style annotation saved to:\n{output_json_path}")



✅ COCO-style annotation saved to:
D:\School\Semester 8\PENGANTAR DEEP LEARNING\UTS\dataset-F\annotations.json
